In [ ]:
## $lookup (agregação)

#Executa uma união externa esquerda em uma collection no mesmo banco de dados para filtrar documentos da collection externa para processamento. 
# #O estágio $lookup adiciona um novo campo de array a cada documento de entrada. 
# #O novo campo da matriz contém os documentos correspondentes da collection estrangeira.

In [ ]:
#colecao clientes

[
  { "_id": 1, "cliente_id": "C001", "nome": "João", "estado": "SC" },
  { "_id": 2, "cliente_id": "C002", "nome": "Maria", "estado": "SP" },
  { "_id": 3, "cliente_id": "C003", "nome": "Carlos", "estado": "SC" },
  { "_id": 4, "cliente_id": "C004", "nome": "Ana", "estado": "RJ" }
]

In [ ]:
#Coleção Pedidos

[
  {
    "_id": 1001,
    "cliente_id": "C001",
    "data": "2024-12-01",
    "estado": "SC",
    "produtos": [
      { "produto": "Notebook", "preco": 3500, "quantidade": 1 },
      { "produto": "Mouse", "preco": 50, "quantidade": 2 }
    ],
    "status": "entregue"
  },
  {
    "_id": 1002,
    "cliente_id": "C001",
    "data": "2025-01-15",
    "estado": "PR",
    "produtos": [
      { "produto": "Teclado", "preco": 120, "quantidade": 1 }
    ],
    "status": "pendente"
  },
  {
    "_id": 1003,
    "cliente_id": "C003",
    "data": "2025-02-10",
    "estado": "SP",
    "produtos": [
      { "produto": "Monitor", "preco": 1100, "quantidade": 1 }
    ],
    "status": "entregue"
  },
  {
    "_id": 1004,
    "cliente_id": "C004",
    "data": "2025-02-15",
    "estado": "SC",
    "produtos": [
      { "produto": "Impressora", "preco": 900, "quantidade": 1 }
    ],
    "status": "cancelado"
  }
]

In [ ]:
#Exemplo de consulta simples sem variaveis com $match + $lookup

[
  {
    "$match": { "estado": "SC" }
  },
  {
    "$lookup": {
      "from": "pedidos",
      "localField": "cliente_id",
      "foreignField": "cliente_id",
      "as": "pedidos_cliente"
    }
  }
]


In [ ]:
###   $let

#A cláusula let em um $lookup no MongoDB permite passar variáveis do documento externo (da coleção principal) para o pipeline interno (da coleção secundária). 
#Isso te dá muito mais poder para criar joins com condições complexas.

{
  "$lookup": {
    "from": "pedidos",
    "let": { "cid": "$cliente_id" },
    "pipeline": [
      {
        "$match": {
          "$expr": {
            "$and": [
              { "$eq": ["$cliente_id", "$$cid"] },
              { "$eq": ["$status", "entregue"] }
            ]
          }
        }
      }
    ],
    "as": "pedidos_entregues"
  }
}

✅#$lookup Inicia uma operação de junção (join) com outra coleção.
✅#"from": "pedidos" :Nome da coleção secundária que será consultada.Neste caso, vamos buscar dados da coleção pedidos.
✅#let: { "cid": "$cliente_id" } Cria uma variável temporária chamada cid que recebe o valor do campo cliente_id do documento atual da coleção principal (ex: clientes).
✅# "pipeline": [...] Define o pipeline de agregação que será executado sobre a coleção pedidos, usando a variável cid.

In [ ]:
##   $lookup com mais de uma variavel no $let

[
  {
    "$lookup": {
      "from": "pedidos",
      "let": {
        "cid": "$cliente_id",
        "uf": "$estado"
      },
      "pipeline": [
        {
          "$match": {
            "$expr": {
              "$and": [
                { "$eq": ["$cliente_id", "$$cid"] },
                { "$eq": ["$estado", "$$uf"] },
                { "$eq": ["$status", "entregue"] }
              ]
            }
          }
        }
      ],
      "as": "pedidos_entregues"
    }
  }
]


In [ ]:
## lookup com project

[
  {
    "$lookup": {
      "from": "pedidos",
      "let": { "cid": "$cliente_id" },
      "pipeline": [
        {
          "$match": {
            "$expr": {
              "$and": [
                { "$eq": ["$cliente_id", "$$cid"] },
                { "$eq": ["$status", "entregue"] }
              ]
            }
          }
        },
        {
          "$project": {
            "_id": 0,
            "data": 1,
            "status": 1,
            "total_pedido": {
              "$sum": {
                "$map": {
                  "input": "$produtos",
                  "as": "item",
                  "in": { "$multiply": ["$$item.preco", "$$item.quantidade"] }
                }
              }
            }
          }
        }
      ],
      "as": "pedidos_entregues"
    }
  }
]


# ✅ $map: itera sobre o array produtos do pedido.

# ✅ as: "item": cada elemento do array será referenciado como $$item.

# ✅ $multiply: multiplica preco * quantidade de cada item.

# ✅ $sum: soma os totais de cada item para gerar o total geral do pedido.

In [ ]:
##   $lookup entre bancos diferentes
#forma simples
{
  "$lookup": {
    "from": "vendas.pedidos",   #// <-- banco.coleção
    "localField": "cliente_id",
    "foreignField": "cliente_id",
    "as": "pedidos_do_cliente"
  }
}



## usando let + pipeline
{
  "$lookup": {
    "from": "vendas.pedidos2",
    "let": { "cid": "$cliente_id" },
    "pipeline": [
      {
        "$match": {
          "$expr": {
            "$eq": ["$cliente_id", "$$cid"]
          }
        }
      }
    ],
    "as": "pedidos_do_cliente"
  }
}




#⚠️ Importante:Isso só funciona se os bancos estiverem no mesmo cluster (ou seja, no mesmo servidor).
#No MongoDB Compass, você também pode rodar esse tipo de agregação normalmente, desde que ambos os bancos estejam visíveis.

In [ ]:
from pymongo import MongoClient
import pprint

# Conecta ao MongoDB
client = MongoClient("mongodb://localhost:27017/")

# Define os bancos e coleções
db1 = client["Joins"]
cliente = db1["cliente"]

db2 = client["vendas"]
pedidos2 = db2["pedidos2"]




In [3]:
for doc in pedidos2.find():
    print(doc)

{'_id': 1001000, 'cliente_id': 'C001', 'data': '2024-12-01', 'estado': 'SC', 'produtos': [{'produto': 'Notebook', 'preco': 3500, 'quantidade': 1}, {'produto': 'Mouse', 'preco': 50, 'quantidade': 2}], 'status': 'entregue'}
{'_id': 1002000, 'cliente_id': 'C001', 'data': '2025-01-15', 'estado': 'PR', 'produtos': [{'produto': 'Teclado', 'preco': 120, 'quantidade': 1}], 'status': 'pendente'}
{'_id': 1003000, 'cliente_id': 'C003', 'data': '2025-02-10', 'estado': 'SP', 'produtos': [{'produto': 'Monitor', 'preco': 1100, 'quantidade': 1}], 'status': 'entregue'}
{'_id': 1004000, 'cliente_id': 'C004', 'data': '2025-02-15', 'estado': 'SC', 'produtos': [{'produto': 'Impressora', 'preco': 900, 'quantidade': 1}], 'status': 'cancelado'}


In [4]:
clientes_data = list(cliente.find())


In [ ]:

for cliente in clientes_data:
    cid = cliente["cliente_id"]
    pedidos_do_cliente = list(pedidos2.find({"cliente_id": cid}))
    cliente["pedidos"] = pedidos_do_cliente

    pprint.pprint(clientes_data)

In [6]:
def carregar_clientes_com_pedidos():
    from pymongo import MongoClient
    import pprint

    client = MongoClient("mongodb://localhost:27017/")
    cliente = client["Joins"]["cliente"]
    pedidos2 = client["vendas"]["pedidos2"]

    clientes_data = list(cliente.find())
    for c in clientes_data:
        cid = c["cliente_id"]
        c["pedidos"] = list(pedidos2.find({"cliente_id": cid}))

    pprint.pprint(clientes_data)

In [ ]:
carregar_clientes_com_pedidos()

In [ ]:
## unwind
#📌 Para que ele serve?
# Quando você usa um $lookup, o resultado é um array de documentos (mesmo que só exista um item). 
# O $unwind é necessário quando você quer acessar diretamente os campos desse array, como se fosse um objeto normal — útil para fazer joins mais limpos, 
# filtros ou projeções.

In [ ]:
[
  {
    "$lookup": {
      "from": "pedidos",
      "let": { "cid": "$cliente_id" },
      "pipeline": [
        {
          "$match": {
            "$expr": {
              "$and": [
                { "$eq": ["$cliente_id", "$$cid"] },
                { "$eq": ["$status", "entregue"] }
              ]
            }
          }
        }
      ],
      "as": "pedidos_entregues"
    }
  },
  {
    "$unwind": "$pedidos_entregues"
  }
]

In [ ]:
[
  {
    "$lookup": {
      "from": "pedidos",
      "let": { "cid": "$cliente_id" },
      "pipeline": [
        {
          "$match": {
            "$expr": {
              "$eq": ["$cliente_id", "$$cid"]
            }
          }
        }
      ],
      "as": "pedidos_entregues"
    }
  },
  {
    "$unwind": "$pedidos_entregues"
  },
  {
    "$unwind": "$pedidos_entregues.produtos"
  },
  {
    "$match": {
      "pedidos_entregues.status": "entregue"
    }
  },
  {
    "$project": {
      "_id": 0,
      "nome": 1,
      "produto": "$pedidos_entregues.produtos.produto",
      "preco": "$pedidos_entregues.produtos.preco",
      "quantidade": "$pedidos_entregues.produtos.quantidade",
      "status": "$pedidos_entregues.status"
    }
  }
]


In [ ]:
#Trabalho

Criar uma estrutura de consultas Medicas conforme exemplo abaixo:

Inlcuir 20 medicos, nas especilidades fazer variações
INlcuir 30 consultas
Incluir 30 remedios
incluir 40 prescrições

Coleção Médicos

{
  "medico_id": "M001",
  "nome": "Dr. Ana Beatriz",
  "crm": "123456-SP",
  "especialidade": "Cardiologia",
  "email": "ana@hospital.com",
  "telefone": "(11) 99999-0000"
}

Coleção Consultas

{
  "consulta_id": "C001",
  "data": "2025-06-15T14:00:00",
  "paciente": {
    "nome": "João da Silva",
    "cpf": "123.456.789-00",
    "nascimento": "1980-05-12"
  },
  "medico_id": "M001",
  "motivo": "Dor no peito",
  "observacoes": "Paciente com histórico de hipertensão."
}

Coleção Remédios

{
  "remedio_id": "R001",
  "nome": "AAS",
  "dosagem": "100mg",
  "forma_farmaceutica": "Comprimido",
  "fabricante": "Bayer"
}

Coleção Prescrições 

{
  "prescricao_id": "P001",
  "consulta_id": "C001",
  "medico_id": "M001",
  "data_prescricao": "2025-06-15T14:30:00",
  "instrucoes_gerais": "Tomar após as refeições.",
  "itens": [
    {
      "remedio_id": "R001",
      "frequencia": "1x ao dia",
      "duracao": 30
    },
    {
      "remedio_id": "R002",
      "frequencia": "2x ao dia",
      "duracao": 60 
    }
  ]
}

In [ ]:
Consultas a serem criadas

1- Total de consultas por médico, trazer os campos nome, especialidade, quantidade de consultas
2-Total de consultas por especialidade, ordenadas por quantidade da maior para a menor quantidade, deve conter nome, especialidade e quantidade.
3-Duração média das prescrições feita por cada medico,deve conter nome, especialidade e média da duração.
4-Remedio Mais receitado, contendo nome e quantidade de prescrições.
5-Trazer consultas efetuadas pelos médicos com prescrições e remédios, deverá conter, Nome do médico, data da consulta, nome de remédios prescritos, dosagem, frequência e duração.

